## Imports

In [1]:
from ipynb.fs.full.Jacobian import *
from ipynb.fs.full.SubFunctions import *
from ipynb.fs.full.GlobalVars import *

## Solver for one differencial equation

In [2]:
def Solver_one_equation(expr, u0, use_trapezoidal, symbol):
    t0 = 0
    i = 1
    f_uk = 1
    uk_1 = u0
    uk_global = u0
    nb = 0
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Function = [u0, u1, u2, ...., un]
    function = [u0]
    time = [t0]

    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        #Newton's method
        while(nb < Nnewton):
            
            # First step, we change our equation to f(x) = 0
            f = create_f(symbol, uk_global, expr, use_trapezoidal)
            print("f = ", end='')
            print(f)
            
            # We recup the term f(uk-1)
            print(symbol)
            print(uk_1)
            f_uk_1 = float(f.subs(symbol, uk_1))
    
            
            #We recup the term f'(uk_1)
            f_uk_1_prime = Jacobian_one(f).subs(symbol, uk_1)

            #Formula : uk = uk_1 - (f(uk_1) / f'(uk_1))
            uk = float(uk_1 - (f_uk_1 / f_uk_1_prime))

            print("fuk_1 = ", end='')
            print(f_uk_1)
            print("fprime = ", end='')
            print(f_uk_1_prime)
            print("uk = ", end='')
            print(uk)
            
            
            
            #Check the approximation. If f_uk < Epsilon = good approximation
            f_uk = f.subs(symbol, uk)
            norm_uk = LA.norm(uk - uk_global)
            print("f_uk = ", end='')
            print(f_uk)
            print("norm_uk =", end='')
            print(norm_uk)
            
            uk_1 = uk
            if(f_uk < Epsilon or norm_uk < Epsilon):
                break
        
        print("--------")
        #Update our 2 lists and indexes.
        time.append(t0)
        function.append(uk)
        uk_global = uk
        f_uk = 1
        i += 1
        nb += 1

    return result_solver(time, function)

## Solver for 2 differentials equations

In [3]:
def Solver_two_equations(u0, v0, f1, f2, use_trapezoidal):
    t0 = 0
    f_xk = 1
    uk_1 = u0
    vk_1 = v0
    xk_1 = np.array([u0, v0])
    xn_1_global = np.array([u0, v0])
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            # First step, we change our equation to F(x) = 0
            vec_f = create_2_f(xn_1_global, f1, f2, use_trapezoidal)
            
            #Creating F(xk-1)
            F_xk_1 = create_fxk_1(xk_1, vec_f)
            
            #Creating F'(xk-1)
            F_xk_1_prime = f_prime_2_eqs(xk_1, vec_f[0], vec_f[1])
        
            
            #Formula : xk = xk_1 - tr(F'(xk_1)) * F(xk_1)
            xk = xk_1 - np.dot(LA.inv(F_xk_1_prime), F_xk_1)
          
            #2 types of check the approximation.
            f_xk =[0, 0]
            f_xk[0] = float(vec_f[0].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[1] = float(vec_f[1].evalf(subs={u: xk[0], v: xk[1]})) 
            norm_f_xk = LA.norm(f_xk)
            norm_xk = LA.norm(xk - xn_1_global)
            if(norm_xk < Epsilon or norm_f_xk < Epsilon):
                break
            
            xk_1 = xk
        
        #Update our 3 lists and indexes.
        time.append(t0)
        fu.append(xk[0])
        fv.append(xk[1])
        
        xn_1_global = np.array([xk[0],xk[1]])
        f_xk = 1        
      
    return result_solver(time, fu, fv)

## Solver for 4 differentials equations

In [4]:
def Solver_4_equations(u0, v0, f1, f2, use_trapezoidal):
    t0 = 0
    f_xk = 1
    uk_1 = u0
    vk_1 = v0
    xk_1 = np.array([u0, v0])
    xn_1_global = np.array([u0, v0])
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Suit of x and y variables
    y0 = u0*v0
    x0 = u0 + v0 - y0
    x = [x0]
    y = [y0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            # First step, we change our equation to F(x) = 0
            vec_f = create_2_f(xn_1_global, f1, f2, use_trapezoidal)
            
            #Creating F(xk-1)
            F_xk_1 = create_fxk_1(xk_1, vec_f)
            
            #Creating F'(xk-1)
            F_xk_1_prime = f_prime_2_eqs(xk_1, vec_f[0], vec_f[1])
        
            #Formula : xk = xk_1 - tr(F'(xk_1)) * F(xk_1)
            xk = xk_1 - np.dot(LA.inv(F_xk_1_prime), F_xk_1)
            
            #With the uk+1 and vk+1 calculated, we can calcul the x and y value.
            next_y = xk[0]*xk[1]
            next_x = xk[0] + xk[1] - next_y
            
            #Calcul of f(uk, vk, xk, yk) for the approximation
            f_xk = [0, 0, 0, 0]
            f_xk[0] = float(vec_f[0].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[1] = float(vec_f[1].evalf(subs={u: xk[0], v: xk[1]}))
            f_xk[2] = float(next_x)
            f_xk[3] = float(next_y)
            
            #2 types of check the approximation.
            norm_f_xk = LA.norm(f_xk)
            norm_xk = LA.norm(xk - xn_1_global)
            
            if(norm_xk < Epsilon or norm_f_xk < Epsilon):
                break
            
            xk_1 = xk
        
        #Update our 2 functions + time
        time.append(t0)
        fu.append(xk[0])
        fv.append(xk[1])
        
        #Update x and y variables
        x.append(next_x)
        y.append(next_y)
            
        #Update indexes
        xn_1_global = np.array([xk[0],xk[1]])
        f_xk = 1        
      
    return result_solver(time, fu, fv, x, y)